Проективное преобразование


In [4]:
from names import WHITE, DEER, RABBIT, FOX, STORM_TROOPER
from type import Scale
from core import ObjModel, CustomImage, init_z_buffer, face_angle_cos, intensity_color, adapt_triangle_to_image, suitable_pixels, barisentrik_coordinates


def draw_faces(obj: ObjModel, _img: CustomImage, scale: Scale):
    z_buffer = init_z_buffer(_img)

    for f in obj.faces():
        cos = face_angle_cos(f)
        if cos > 0:
            continue

        color = intensity_color(WHITE, cos)

        z0, z1, z2 = f[0][2], f[1][2], f[2][2]
        f = adapt_triangle_to_image(f, _img, scale)

        for v in suitable_pixels(f, _img):
            i, j = v
            l0, l1, l2 = barisentrik_coordinates(v, f)
            if l0 >= 0 and l1 >= 0 and l2 >= 0:
                z = l0 * z0 + l1 * z1 + l2 * z2
                if z < z_buffer[i][j]:
                    z_buffer[i][j] = z
                    _img.set(v, color)


deer_obj = ObjModel(DEER)
rabbit_obj = ObjModel(RABBIT)
fox_obj = ObjModel(FOX)
storm_trooper_obj = ObjModel(STORM_TROOPER)

img = CustomImage(1500, 1500)
deer_obj.shift((0, -800, 1000))
draw_faces(deer_obj, img, (700, 700))
img.save('pictures/faces/deer_projection.png')

img = CustomImage(1000, 1000)
rabbit_obj.shift((0.005, -0.045, 15.0))
draw_faces(rabbit_obj, img, (100000, 100000))
img.save('pictures/faces/rabbit_projection.png')

img = CustomImage(1000, 800)
fox_obj.shift((0, -40, 1000))
draw_faces(fox_obj, img, (6000, 6000))
img.save('pictures/faces/fox_projection.png')

img = CustomImage(1500, 1500)
storm_trooper_obj.shift((0, 0, 100))
draw_faces(storm_trooper_obj, img, (30000, 30000))
img.save('pictures/faces/storm_trooper_projection.png')

Поворот модели

In [8]:
import math
from names import WHITE, DEER, RABBIT, FOX, STORM_TROOPER
from type import Scale
from core import ObjModel, CustomImage, init_z_buffer, face_angle_cos, intensity_color, adapt_triangle_to_image, suitable_pixels, barisentrik_coordinates


def draw_faces(obj: ObjModel, _img: CustomImage, scale: Scale):
    z_buffer = init_z_buffer(_img)

    for f in obj.faces():
        cos = face_angle_cos(f)
        if cos > 0:
            continue

        color = intensity_color(WHITE, cos)

        z0, z1, z2 = f[0][2], f[1][2], f[2][2]
        f = adapt_triangle_to_image(f, _img, scale)

        for v in suitable_pixels(f, _img):
            i, j = v
            l0, l1, l2 = barisentrik_coordinates(v, f)
            if l0 >= 0 and l1 >= 0 and l2 >= 0:
                z = l0 * z0 + l1 * z1 + l2 * z2
                if z < z_buffer[i][j]:
                    z_buffer[i][j] = z
                    _img.set(v, color)


deer_obj = ObjModel(DEER)
rabbit_obj = ObjModel(RABBIT)
fox_obj = ObjModel(FOX)
storm_trooper_obj = ObjModel(STORM_TROOPER)

img = CustomImage(1500, 1500)
deer_obj.rotate((math.pi, 0, 0))
deer_obj.shift((0, 800, 1000))
draw_faces(deer_obj, img, (700, 700))
img.save('pictures/faces/deer_angle.png')

img = CustomImage(1000, 1000)
rabbit_obj.rotate((math.pi, 0, 0))
rabbit_obj.shift((0.005, 0.045, 15.0))
draw_faces(rabbit_obj, img, (100000, 100000))
img.save('pictures/faces/rabbit_angle.png')

img = CustomImage(1200, 800)
fox_obj.rotate((math.pi, math.pi / 2, 0))
fox_obj.shift((0, 40, 1000))
draw_faces(fox_obj, img, (6000, 6000))
img.save('pictures/faces/fox_angle.png')

img = CustomImage(1500, 1500)
storm_trooper_obj.rotate((math.pi, 0, 0))
storm_trooper_obj.shift((0, 0, 100))
draw_faces(storm_trooper_obj, img, (30000, 30000))
img.save('pictures/faces/storm_trooper_angle.png')